In [20]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
from sklearn.cluster import AgglomerativeClustering
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import numpy as np
import hdbscan
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

/opt/anaconda3/envs/LLJSpectrum/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[nltk_data] Downloading package stopwords to /Users/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/matt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
dfllj_hdb= pd.read_csv('/Users/matt/Desktop/LLJ-Data/LLJSpectrum/YearAlteredData/dflljyear.csv')
dfspec_hdb= pd.read_csv('/Users/matt/Desktop/LLJ-Data/LLJSpectrum/YearAlteredData/dfspecyear.csv')

In [22]:
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    
    text = text.lower()
    
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    
    tokens = text.split()
    
    
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    
    text = ' '.join(tokens)
    
    return text

In [23]:
dfllj_hdb['Title'] = dfllj_hdb['Title'].apply(preprocess_text)
dfspec_hdb['Title'] = dfspec_hdb['Title'].apply(preprocess_text)

In [24]:
dfllj_hdb['Title'] = dfllj_hdb['Title'].astype(str)
embeddingsllj = model.encode(dfllj_hdb['Title'].tolist())
dfllj_hdb['Embeddings'] = list(embeddingsllj)

In [25]:
dfspec_hdb['Title'] = dfspec_hdb['Title'].astype(str)
embeddingsspec = model.encode(dfspec_hdb['Title'].tolist())
dfspec_hdb['Embeddings'] = list(embeddingsspec)

In [32]:
embeddings_array_llj = np.array(dfllj_hdb['Embeddings'].tolist())
clustererhdb = hdbscan.HDBSCAN(min_cluster_size=3, min_samples= 2, metric='euclidean')
cluster_labels_llj = clustererhdb.fit_predict(embeddings_array_llj)
dfllj_hdb['Cluster'] = cluster_labels_llj

In [33]:
print(cluster_labels_llj)

[-1 81 -1 ... 17 16 -1]


In [35]:
num_clusters = len(set(cluster_labels_llj)) - (1 if -1 in cluster_labels_llj else 0)
print(f'Number of clusters: {num_clusters}')

Number of clusters: 102


In [36]:
for cluster in set(cluster_labels_llj):
    if cluster != -1:  # -1 is the noise label
        print(f'\nCluster {cluster}:')
        cluster_titles = dfllj_hdb[dfllj_hdb['Cluster'] == cluster]['Title'].tolist()
        for title in cluster_titles:
            print(title)


Cluster 0:
evaluating shepard keycite bcite case validation accuracy
comparing keycite shepard completeness currency accuracy
response taylor comparison shepard keycite
another response taylor comparison keycite shepard

Cluster 1:
unanswerable question
question answer
question answer
question answer
question answer
question answer
question answer
question answer
reply herskowitzs unanswered question
asking help

Cluster 2:
right receive information
must talk information common
manual inform every citizen

Cluster 3:
sketch life william j c berry
st george tuckera biographical sketch
john norton pomeroy biographical sketch

Cluster 4:
keeping new legal title
keeping new legal title
keeping new legal title
keeping new legal title
keeping new legal title
keeping new legal title

Cluster 5:
introductory statement
introduction
part introductory remark
introduction
introduction
introduction
introduction
introduction
introduction

Cluster 6:
keeping new legal title
keeping new legal title
k

In [30]:
dfllj_hdb[dfllj_hdb['Cluster'] == -1]['Title'].tolist()

['address joint session american library association',
 'legislative reference work law library',
 'cataloguing law book special reference cooperative indexing index card',
 'coke report anonymous commentary littleton',
 'byproduct worcester county law library',
 'recent development international municipal law',
 'need college instruction use law book',
 'law legislative reference part state library',
 'preserving heritage protecting law library core mission updated library quality assessment standard',
 'discovering william cook ten source reconstructing life lawyer',
 'form follow function academic law library organizational structure collection development',
 'choosing top candidate best practice academic law library hiring',
 'essential law library journal',
 'law librarian jd jd',
 'office email burden blessing',
 'fuss library',
 'robert l oakley memoriam',
 'georgetown law center memorial service november',
 'remembering bob oakley',
 'en la tierra del ciego el tuerco e rey prob